# Install Libraries

In [1]:
!pip install selenium

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


# Imports

In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from time import sleep
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

# Work

In [2]:
base_url = 'http://diariooficial.rn.gov.br/dei/dorn3/'

binary = r'C:\Program Files\Mozilla Firefox\firefox.exe'
options = Options()
options.headless = True
options.binary = binary

cap = DesiredCapabilities().FIREFOX
cap["marionette"] = True #optional

driver = webdriver.Firefox(options=options, capabilities=cap)
#driver = webdriver.Firefox(capabilities=cap)
driver.get(base_url)

In [3]:
materias = []

In [4]:
data = driver.find_element_by_class_name('ui-datepicker-days-cell-over.ui-datepicker-current-day')
ano = data.get_attribute('data-year')
mes = data.get_attribute('data-month')
dia = data.find_element_by_tag_name('a').get_attribute('text')
data_formatada = (dia + '/0' + mes + '/' + ano if int(mes) < 10 else dia + '/' + mes + '/' + ano)
data_formatada = ('0' + data_formatada if int(dia) < 10 else data_formatada)
data_formatada

'03/09/2019'

In [5]:
def processar_linhas(link, text, blank):
    caminho = driver.find_element_by_id("lblSessaoPath").text
    lista_caminho = caminho.split("\\")[:-1]
    if blank != 0:
        [lista_caminho.append('') for b in range(blank)]
    if lista_caminho[0] == "DIÁRIO DA JUSTIÇA":
        lista_caminho.insert(0,'DIÁRIO OFICIAL')
    lista_caminho.insert(0, data_formatada)
    lista_caminho.append(text)
    lista_caminho.append(link)
    print(lista_caminho)
    return lista_caminho

In [6]:
def navegar(link, blank=5):
    link.click()
    global materias
    subsessoes = []
    try:
        subsessoes = driver.find_element_by_id("lblQtdMateriasPublicadas")
        if subsessoes.text == 'Nenhuma matéria encontrada nessa data para essa seção.':
            materia = processar_linhas(link=subsessoes.text, text='Não Possui', blank=blank)
            materias.append(materia)
            return []
        tabela = driver.find_element_by_id("dgDocumentos")
        files = tabela.find_elements_by_tag_name("a")
        for f in files:
            link = f.get_attribute("href")
            materia = processar_linhas(link=link, text=f.text, blank=blank)
            materias.append(materia)
        try:
            tabela = driver.find_element_by_id("dgSubSessoes")
            links = tabela.find_elements_by_tag_name("a")
            return links
        except:
            return []
    except:
        tabela = driver.find_element_by_id("dgSubSessoes")
        links = tabela.find_elements_by_tag_name("a")
    return links

#elements = driver.find_element_by_id("Menu1_dgMenu_ctl04_LinkButton1")
#teste = navegar(elements)
#materias

In [7]:
def crawl_diario(link_sessao):
    #sleep(1)
    sub1 = navegar(link_sessao)
    if len(sub1) != 0:
        id1 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" if i < 10 
               else "dgSubSessoes_ctl"+str(i)+"_Linkbutton1" for i in range(2,len(sub1)+2)]
        for i1 in id1:
            #sleep(1)
            #print(i1)
            s1 = driver.find_element_by_id(i1)
            sub2 = navegar(s1, blank=4)
            if len(sub2) == 0:
                #sleep(1)
                driver.back()
            else:
                id2 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" if i < 10 
                       else "dgSubSessoes_ctl"+str(i)+"_Linkbutton1" for i in range(2,len(sub2)+2)]
                for i2 in id2:
                    #sleep(1)
                    s2 = driver.find_element_by_id(i2)
                    sub3 = navegar(s2, blank=3)
                    if len(sub3) == 0:
                        #sleep(1)
                        driver.back()
                    else:
                        id3 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" if i < 10 
                               else "dgSubSessoes_ctl"+str(i)+"_Linkbutton1" for i in range(2,len(sub3)+2)]
                        for i3 in id3:
                            #sleep(1)
                            s3 = driver.find_element_by_id(i3)
                            sub4 = navegar(s3, blank=2)
                            if len(sub4) == 0:
                                #sleep(1)
                                driver.back()
                            else:
                                id4 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" if i < 10 
                                       else "dgSubSessoes_ctl"+str(i)+"_Linkbutton1" for i in range(2,len(sub4)+2)]
                                for i4 in id4:
                                    #sleep(1)
                                    s4 = driver.find_element_by_id(i4)
                                    sub5 = navegar(s4, blank=1)
                                    if len(sub5) == 0:
                                        #sleep(1)
                                        driver.back()
                                    else:
                                        id5 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" if i < 10 
                                               else "dgSubSessoes_ctl"+str(i)+"_Linkbutton1" for i in range(2,len(sub5)+2)]
                                        for i5 in id5:
                                            #sleep(1)
                                            s5 = driver.find_element_by_id(i5)
                                            sub6 = navegar(s5, blank=0)
                                            if len(sub6) == 0:
                                                #sleep(1)
                                                driver.back()
                                            if i5 == id5[-1]:
                                                #sleep(1)
                                                driver.back()
                                    if i4 == id4[-1]:
                                        #sleep(1)
                                        driver.back()
                            if i3 == id3[-1]:
                                #sleep(1)
                                driver.back()
                    if i2 == id2[-1]:
                        #sleep(1)
                        driver.back()
            if i1 == id1[-1]:
                #sleep(1)
                driver.back()

segunda versão

In [8]:
elements = ["Menu1_dgMenu_ctl0" + str(i) + "_LinkButton1" for i in range(2,10)]
#lista = navegar(sessoes[7])
#lista = []
for e in elements:
    s = driver.find_element_by_id(e)
    crawl_diario(s)
print(materias)
#lista

['03/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (ASSECOM)', 'DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI)', 'DEI ASSEJUR', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'CONSELHO DE DESENVOLVIMENTO DO ESTADO (CDE)', '', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'CONSULTORIA GERAL DO ESTADO (CGE)', '', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'CONTROLADORIA GERAL DO ESTADO (CONTROL)', '', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'DEFENSORIA PÚBL

['03/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE ESTADO DA AGRICULTURA, DA PECUÁRIA E DA PESCA ', 'CENTRAIS DE ABASTECIMENTO DO RN (CEASA)', 'PROGRAMA FARMACIA DE TODOS -CEASA/RN', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE ESTADO DA AGRICULTURA, DA PECUÁRIA E DA PESCA ', 'EMPRESA DE PESQUISA AGROPECUÁRIA DO RN (EMPARN)', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE ESTADO DA AGRICULTURA, DA PECUÁRIA E DA PESCA ', 'INSTITUTO DE ASSISTÊNCIA TÉCNICA E EXTENSÃO RURAL DO RN (EMATER)', '', '', '57ª Convocação - Edital 002/2016', 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20191003&id_doc=660952']
['03/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CON

['03/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE ESTADO DO PLANEJAMENTO E DAS FINANÇAS (SEPLAN)', '', '', '', 'SEPLAN - PUB. 02.10.2019 - TERMO DE HOMOLOGAÇÃO E INEXIGIBILIDADE.doc', 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20191003&id_doc=661068']
['03/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE ESTADO DO PLANEJAMENTO E DAS FINANÇAS (SEPLAN)', 'AGÊNCIA REGULADORA DE SERVIÇOS PÚBLICOS DO RN (ARSEP)', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE ESTADO DO PLANEJAMENTO E DAS FINANÇAS (SEPLAN)', 'COORDENADORIA DE ESTUDOS SOCIOECONÔMICOS (CES)', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE ESTADO D

['03/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE TRIBUTAÇÃO', 'SECRETARIA DE TRIBUTAÇÃO/SUCIVA', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DE TURISMO', 'EMPRESA POTIGUAR DE PROMOÇÃO TURISTICA S.A-EMPROTUR', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DO DESENVOLVIMENTO ECONÔMICO', 'AGÊNCIA DE FOMENTO DO RN( AGN )', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'CONTRATOS, EDITAIS E AVISOS', 'SECRETARIA DO DESENVOLVIMENTO ECONÔMICO', 'COMPANHIA POTIGUAR DE GÁS (POTIGAS)', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECU

['03/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'SECRETARIAS DE ESTADO', 'SECRETARIA DA JUSTIÇA E DA CIDADANIA ', 'CONSELHO ESTADUAL DE DIREITOS HUMANOS E CIDADANIA', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'SECRETARIAS DE ESTADO', 'SECRETARIA DA SEGURANÇA PÚBLICA E DA DEFESA SOCIAL ', 'ACADEMIA DE POLICIA CIVIL- ACADEPOL', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'SECRETARIAS DE ESTADO', 'SECRETARIA DA SEGURANÇA PÚBLICA E DA DEFESA SOCIAL ', 'CORPO DE BOMBEIROS MILITAR ', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'PODER EXECUTIVO', 'SECRETARIAS DE ESTADO', 'SECRETARIA DA SEGURANÇA PÚBLICA E DA DEFESA SOCIAL ', 'CORREGEDORIA GERAL (SESED)', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019'

...

['03/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'DISTRITO JUDICIÁRIO DA ZONA NORTE - FORO VARELLA BARCA ', 'VARAS CRIMINAIS ', '2ª VARA CRIMINAL ', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'DISTRITO JUDICIÁRIO DA ZONA NORTE - FORO VARELLA BARCA ', 'VARAS CRIMINAIS ', '3ª VARA CRIMINAL ', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'DISTRITO JUDICIÁRIO DA ZONA NORTE - FORO VARELLA BARCA ', 'VARAS CRIMINAIS ', '4ª VARA CRIMINAL ', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'DISTRITO JUDICIÁRIO DA ZONA NORTE - FORO VARELLA BARCA ', 'VARAS CRIMINAIS ', '5ª VARA CRIM

['03/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CÍVEIS', 'ESPECIALIZADAS', '1ª VARA DA INFÂNCIA E DA JUVENTUDE ', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CÍVEIS', 'ESPECIALIZADAS', '1ª VARA DE EXECUCAO FISCAL MUNICIPAL E TRIBUTARIA', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CÍVEIS', 'ESPECIALIZADAS', '1ª VARA DE FAMÍLIA ', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CÍVEIS', 'ESPECIALIZADAS', '1ª VARA DE FAZENDA PÚBLICA ', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'DIÁR

['03/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CÍVEIS', 'NÃO ESPECIALIZADAS ', '6ª VARA CÍVEL', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CÍVEIS', 'NÃO ESPECIALIZADAS ', '7ª VARA CÍVEL', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CÍVEIS', 'NÃO ESPECIALIZADAS ', '8ª VARA CÍVEL', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL ', 'VARAS CÍVEIS', 'NÃO ESPECIALIZADAS ', '9ª VARA CÍVEL', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA ESTADUAL ', 'COMARCA DA CAPITAL '

['03/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA FEDERAL DE PRIMEIRO GRAU', '7ª VARA - JUIZADO ESPECIAL FEDERAL - DR. MANUEL MAIA DE VASCONCELOS NETO', '', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA FEDERAL DE PRIMEIRO GRAU', '8ª VARA DA JUSTIÇA FEDERAL -DR. NEWTON FLADSTONE BARBOSA DE MOURA - MOSSORÓ', '', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA FEDERAL DE PRIMEIRO GRAU', '9ª VARA DA JUSTIÇA FEDERAL DR. CARLOS WAGNER DIAS FERREIRA - CAICÓ', '', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA FEDERAL DE PRIMEIRO GRAU', 'DISTRIBUIÇÃO', '', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'JUSTIÇA 

['03/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL DO TRABALHO (TRT)', 'ATOS DAS VARAS DO TRABALHO', '3ª VARA DO TRABALHO DE NATAL', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL DO TRABALHO (TRT)', 'ATOS DAS VARAS DO TRABALHO', '4ª VARA DO TRABALHO DE NATAL', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL DO TRABALHO (TRT)', 'ATOS DAS VARAS DO TRABALHO', '5ª VARA DO TRABALHO DE NATAL', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBUNAL REGIONAL DO TRABALHO (TRT)', 'ATOS DAS VARAS DO TRABALHO', '6ª VARA DO TRABALHO DE NATAL', '', '', 'Não Possui', 'Nenhuma matéria encontrada nessa data para essa seção.']
['03/09/2019', 'DIÁRIO OFICIAL', 'DIÁRIO DA JUSTIÇA', 'TRIBU

In [9]:
driver.quit()

In [10]:
df = pd.DataFrame(materias,columns=['data_publicacao', 'diario', 'sessao', 'subsessao1', 'subsessao2', 'subsessao3', 'subsessao4', 'subsessão5', 'nome_materia', 'materia'])
df.head()

,data_publicacao,diario,sessao,subsessao1,subsessao2,subsessao3,subsessao4,subsessão5,nome_materia,materia
0,03/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",ASSESSORIA DE COMUNICAÇÃO SOCIAL DO ESTADO (AS...,DEPARTAMENTO ESTADUAL DE IMPRENSA (DEI),DEI ASSEJUR,,Não Possui,Nenhuma matéria encontrada nessa data para ess...
1,03/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",CONSELHO DE DESENVOLVIMENTO DO ESTADO (CDE),,,,Não Possui,Nenhuma matéria encontrada nessa data para ess...
2,03/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",CONSULTORIA GERAL DO ESTADO (CGE),,,,Não Possui,Nenhuma matéria encontrada nessa data para ess...
3,03/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",CONTROLADORIA GERAL DO ESTADO (CONTROL),,,,Não Possui,Nenhuma matéria encontrada nessa data para ess...
4,03/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",DEFENSORIA PÚBLICA DO ESTADO (DPE),,,,Não Possui,Nenhuma matéria encontrada nessa data para ess...


In [11]:
df[df['nome_materia'] == 'Não Possui'].diario.count()

305

In [12]:
df.shape

(433, 10)

In [13]:
filename = "diario - {}".format(data_formatada.replace('/','_'))
df.to_csv(filename)

# Pegar texto das matérias

In [44]:
materias_df = df[df['nome_materia'] != 'Não Possui'].copy()
materias_df.head()

,data_publicacao,diario,sessao,subsessao1,subsessao2,subsessao3,subsessao4,subsessão5,nome_materia,materia
7,03/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",PROCURADORIA GERAL DO ESTADO (PGE),,,,Termo,http://diariooficial.rn.gov.br/dei/dorn3/docvi...
8,03/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,,,,EXTRATO DO QUARTO TERMO ADITIVO AO CONTRATO N°...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...
9,03/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,,,,AVISO RESULTADO DA LICITAÇÃO LICITAÇÃO TOMADA ...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...
16,03/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETÁRIA DE ESTADO DA ADMINISTRAÇÃO (SEAD),INSTITUTO DE PREVIDÊNCIA DOS SERVIDORES DO ESTADO,,,COMPRA DE MATERIAL DE CONSUMO - EXPEDIENTE,http://diariooficial.rn.gov.br/dei/dorn3/docvi...
17,03/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETÁRIA DE ESTADO DA ADMINISTRAÇÃO (SEAD),INSTITUTO DE PREVIDÊNCIA DOS SERVIDORES DO ESTADO,,,"COMPRA DE MATERIAL DE CONSUMO - HIGIENE, LIMPE...",http://diariooficial.rn.gov.br/dei/dorn3/docvi...


In [45]:
print(materias_df.materia.head())

7     http://diariooficial.rn.gov.br/dei/dorn3/docvi...
8     http://diariooficial.rn.gov.br/dei/dorn3/docvi...
9     http://diariooficial.rn.gov.br/dei/dorn3/docvi...
16    http://diariooficial.rn.gov.br/dei/dorn3/docvi...
17    http://diariooficial.rn.gov.br/dei/dorn3/docvi...
Name: materia, dtype: object


In [108]:
def pegar_frame(link):
    r = requests.get(link)
    html_doc = r.text
    soup = BeautifulSoup(html_doc, "html.parser")
    frame = soup.find_all('frame')[1]
    frame_link = frame.get('src')
    url_materia_oficial = base_url + frame_link
    return url_materia_oficial

In [47]:
base_url = 'http://diariooficial.rn.gov.br/dei/dorn3/'

materias_df['link_materia_oficial'] = materias_df['materia'].apply(pegar_frame)
materias_df.head()

,data_publicacao,diario,sessao,subsessao1,subsessao2,subsessao3,subsessao4,subsessão5,nome_materia,materia,link_materia_oficial
7,03/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",PROCURADORIA GERAL DO ESTADO (PGE),,,,Termo,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...
8,03/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,,,,EXTRATO DO QUARTO TERMO ADITIVO AO CONTRATO N°...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...
9,03/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,,,,AVISO RESULTADO DA LICITAÇÃO LICITAÇÃO TOMADA ...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...
16,03/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETÁRIA DE ESTADO DA ADMINISTRAÇÃO (SEAD),INSTITUTO DE PREVIDÊNCIA DOS SERVIDORES DO ESTADO,,,COMPRA DE MATERIAL DE CONSUMO - EXPEDIENTE,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...
17,03/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETÁRIA DE ESTADO DA ADMINISTRAÇÃO (SEAD),INSTITUTO DE PREVIDÊNCIA DOS SERVIDORES DO ESTADO,,,"COMPRA DE MATERIAL DE CONSUMO - HIGIENE, LIMPE...",http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...


In [106]:
print(materias_df.link_materia_oficial.iloc[11])

http://diariooficial.rn.gov.br/dei/dorn3/documentos/00000001/20191003/661014.htm


In [109]:
def pegar_texto(link):
    global cont
    r = requests.get(link)
    html_doc = r.text
    soup = BeautifulSoup(html_doc, "html.parser")
    #if len(soup.find_all('table')) > 0:
    #    soup('table')[0].decompose()
    conteudo_materia = soup.find_all('body')[0].text.strip()
    #conteudo_materia = re.sub("(<!--.*?-->)", "", noticia, flags=re.DOTALL)
    return conteudo_materia

In [110]:
materias_df['texto'] = materias_df['link_materia_oficial'].apply(pegar_texto)
materias_df.head()

,data_publicacao,diario,sessao,subsessao1,subsessao2,subsessao3,subsessao4,subsessão5,nome_materia,materia,link_materia_oficial,texto
7,03/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",PROCURADORIA GERAL DO ESTADO (PGE),,,,Termo,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,EXTRATO DO PRIMEIRO TERMO ADITIVO AO TERMO DE ...
8,03/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,,,,EXTRATO DO QUARTO TERMO ADITIVO AO CONTRATO N°...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,EXTRATO\r\nDO QUARTO TERMO ADITIVO AO CONTRATO...
9,03/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETARIA DA INFRA-ESTRUTURA,,,,AVISO RESULTADO DA LICITAÇÃO LICITAÇÃO TOMADA ...,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,AVISO\nRESULTADO DA LICITAÇÃO\nLICITAÇÃO\r\nTO...
16,03/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETÁRIA DE ESTADO DA ADMINISTRAÇÃO (SEAD),INSTITUTO DE PREVIDÊNCIA DOS SERVIDORES DO ESTADO,,,COMPRA DE MATERIAL DE CONSUMO - EXPEDIENTE,http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,TERMO DE ADESÃO A ATA DE REGISTRO DE PREÇOS Nº...
17,03/09/2019,DIÁRIO OFICIAL,PODER EXECUTIVO,"CONTRATOS, EDITAIS E AVISOS",SECRETÁRIA DE ESTADO DA ADMINISTRAÇÃO (SEAD),INSTITUTO DE PREVIDÊNCIA DOS SERVIDORES DO ESTADO,,,"COMPRA DE MATERIAL DE CONSUMO - HIGIENE, LIMPE...",http://diariooficial.rn.gov.br/dei/dorn3/docvi...,http://diariooficial.rn.gov.br/dei/dorn3/docum...,TERMO DE ADESÃO A ATA DE REGISTRO DE\r\nPREÇOS...


In [119]:
print(materias_df.texto.iloc[50])

PORTARIA Nº 055/2019-GDG/PCRN, DE 30 DE SETEMBRO DE 2019. 
A DELEGADA-GERAL DE POLÍCIA CIVIL DO ESTADO DO RIO GRANDE DO
NORTE, no uso das atribuições que lhe confere o art. 15, inciso VI, da Lei
Complementar nº 270, de 13/02/2004, 
CONSIDERANDO
que a remoção de pessoal da Polícia Civil poderá ser feita a pedido, nos termos
do art. 81, inciso I, da referida Lei;
CONSIDERANDO
que a autoridade administrativa tem poderes para determinar lotação, designação
ou remoção de servidor, ante ao poder discricionário do poder público, baseado
nos critérios de conveniência e oportunidade;
CONSIDERANDO que as diretrizes da Administração Pública devem
ser traçadas em consonância com os princípios dispostos no caput do artigo 37
da Constituição da República Federativa do Brasil, e que seus atos são
vinculados e direcionados de modo a
garantir os interesses e necessidades da coletividade, em homenagem ao
princípio da supremacia do interesse público,
RESOLVE:
Art.
1º REMOVER CRISTIANO JERÔNIMO BACELAR, A

In [120]:
filename = "materias_diario - {}".format(data_formatada.replace('/','_'))
materias_df.to_csv(filename)

# O que fazer
### Seleção de data
    Iterar por data (opcional)
### Criar Funções (fazendo)
### Minimizar gambiarras
### Criar arquivo .py

# Teste

Pegar subsessões

In [103]:
elements = driver.find_element_by_id("Menu1_dgMenu_ctl02_LinkButton1")
elements.click()
tabela = driver.find_element_by_id("dgDocumentos")
subsessoes = tabela.find_elements_by_tag_name("a")
for s in subsessoes:
    print(s.text)

CONTRATOS, EDITAIS E AVISOS
GOVERNADORIA
SECRETARIAS DE ESTADO


Pegar links html

In [54]:
elements = driver.find_element_by_id("Menu1_dgMenu_ctl04_LinkButton1")
print(elements)
elements.click()
tabela = driver.find_element_by_id("dgDocumentos")
files = tabela.find_elements_by_tag_name("a")
links = [f.get_attribute("href") for f in files]
links

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="00bc14c9-d812-4d3b-9a60-da3689727de3")>


['http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20190926&id_doc=660291',
 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20190926&id_doc=660292',
 'http://diariooficial.rn.gov.br/dei/dorn3/docview.aspx?id_jor=00000001&data=20190926&id_doc=660293']

Caminhar até o fim

In [52]:
elements = driver.find_element_by_id('Menu1_dgMenu')
sessoes = elements.find_elements_by_tag_name('a')

subsessoes_1 = navegar(sessoes[0])
subsessoes_2 = navegar(subsessoes_1[0])
subsessoes_3 = navegar(subsessoes_2[0])
subsessoes_4 = navegar(subsessoes_3[0])
subsessoes_5 = navegar(subsessoes_4[0])
print(subsessoes_5)
driver.back()
driver.back()
driver.back()
#subsessoes_6 = navegar(subsessoes_1[1])

[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="0bb13e6b-92a1-46b8-9527-b68ef5f5c7e1")>, <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="621f55c5-3458-47c2-a793-12f7396cfc6d")>, <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="32f21de6-eeba-4a32-bc08-a1a015bb5532")>, <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="0b66c6b2-57f6-4693-a21a-8d03270e5555")>, <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="80e6c1a8-ea54-4ed6-9e3f-3dc807c79d0c")>, <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="68ea011a-fd01-4355-9e9a-3200c82115c8")>, <selenium.webdriver.firefox.webelement.FirefoxWebElement (sessi

In [53]:
elements = driver.find_element_by_id('Menu1_dgMenu')
elements = driver.find_element_by_id("Menu1_dgMenu_ctl04_LinkButton1")
elements

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="927f94cd-4a14-46bd-b368-9c3c04fe8f0d")>

In [70]:
elementos = ["Menu1_dgMenu_ctl0" + str(i) + "_LinkButton1" for i in range(2,10)]
sessoes = driver.find_elements_by_id(elementos[0])
print(sessoes)
subsessoes_1 = navegar(sessoes[0])
elementos2 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" for i in range(2,len(subsessoes_1)+2)]
print(elementos2)
sessoes2 = driver.find_elements_by_id(elementos2[0])
print(sessoes2)
subsessoes_2 = navegar(sessoes2[0])
elementos3 = ["dgSubSessoes_ctl0"+str(i)+"_Linkbutton1" for i in range(2,len(subsessoes_2)+2)]
print(elementos3)
sessoes3 = driver.find_elements_by_id(elementos3[0])
print(sessoes3)
subsessoes_3 = navegar(sessoes3[0])
subsessoes_4 = navegar(subsessoes_3[0])
subsessoes_5 = navegar(subsessoes_4[0])
print(subsessoes_5)
driver.back()
driver.back()
driver.back()
sessoes6 = driver.find_elements_by_id(elementos3[1])
subsessoes_6 = navegar(sessoes6[0])
print(subsessoes_6)
#baa570e3-c43b-47b4-9cb5-0e844586e406
#dd0912d6-54a2-42fd-a565-5212ce6bdb4d

[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="54215378-e6da-4851-b111-4eb108ad7947")>]
['dgSubSessoes_ctl02_Linkbutton1', 'dgSubSessoes_ctl03_Linkbutton1', 'dgSubSessoes_ctl04_Linkbutton1']
[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="baa570e3-c43b-47b4-9cb5-0e844586e406", element="d08cdc93-a296-4326-9f1a-c8829a1e3736")>]
['dgSubSessoes_ctl02_Linkbutton1', 'dgSubSessoes_ctl03_Linkbutton1', 'dgSubSessoes_ctl04_Linkbutton1', 'dgSubSessoes_ctl05_Linkbutton1', 'dgSubSessoes_ctl06_Linkbutton1', 'dgSubSessoes_ctl07_Linkbutton1', 'dgSubSessoes_ctl08_Linkbutton1', 'dgSubSessoes_ctl09_Linkbutton1', 'dgSubSessoes_ctl010_Linkbutton1', 'dgSubSessoes_ctl011_Linkbutton1', 'dgSubSessoes_ctl012_Linkbutton1', 'dgSubSessoes_ctl013_Linkbutton1', 'dgSubSessoes_ctl014_Linkbutton1', 'dgSubSessoes_ctl015_Linkbutton1', 'dgSubSessoes_ctl016_Linkbutton1', 'dgSubSessoes_ctl017_Linkbutton1', 'dgSubSessoes_ctl018_Linkbutto